# Análisis de componentes principales

En este notebook utilizaremos el análisis de componentes principales con el fin de resumir el comportamiento de 5 índices accionarios en un sólo índice. Este nuevo índice corresponderá al primer componente principal de nuestra matriz de datos

In [1]:
#lee los datos
import pandas as pd
import numpy as np

archivos_csv = ('dow30.csv', 'naftrac.csv', 'nasdaq.csv', 'russell2000.csv', 'sp500.csv')
ruta = 'datos/'

#crea el nombre de las columnas a partir del nombre de cada csv
nombres_columnas = [s.split('.')[0] for s in archivos_csv]

#Se lee el primer archivo csv
#Solo columas Date y Adj Close
columnas = ['Date', 'Adj Close']
datos = pd.read_csv(ruta + archivos_csv[0], usecols = columnas)

#Renombra columna 'Adj Close' (in place)
datos.rename(columns = {'Adj Close': nombres_columnas[0]}, inplace = True)

#Une los datos
for i in range(1, len(archivos_csv)):
    #lee el siguiente archivo csv
    d2 = pd.read_csv(ruta + archivos_csv[i], usecols = columnas)
    
    #Hace un merge utilizando la columna 'Date' como llave
    datos = pd.merge(datos, d2, how = 'inner', on = 'Date', suffixes= ['_' + nombres_columnas[i-1], '_' + nombres_columnas[i]])
    
    #renombra columna 'Adj Close'
    datos.rename(columns = {'Adj Close': nombres_columnas[i]}, inplace = True)

#libera memoria
del d2

#Vista a los primeros 10 renglones
datos.head(10)


,Date,dow30,naftrac,nasdaq,russell2000,sp500
0,2014-08-07,16368.269531,41.218311,4334.970215,1119.760010,1909.569946
1,2014-08-08,16553.929688,41.171616,4370.899902,1131.349976,1931.589966
2,2014-08-11,16569.980469,41.732018,4401.330078,1141.930054,1936.920044
3,2014-08-12,16560.539063,41.769382,4389.250000,1133.030029,1933.750000
4,2014-08-13,16651.800781,41.769382,4434.129883,1141.780029,1946.719971
5,2014-08-14,16713.580078,41.816078,4453.000000,1143.339966,1955.180054
6,2014-08-15,16662.910156,41.666649,4464.930176,1141.650024,1955.060059
7,2014-08-18,16838.740234,41.881470,4508.310059,1158.400024,1971.739990
8,2014-08-19,16919.589844,42.002880,4527.509766,1162.469971,1981.599976
9,2014-08-20,16979.130859,42.236385,4526.479980,1157.510010,1986.510010


In [2]:
#Funcion para calcular los rendimientos
def calcula_rendimientos(datos, nombres_columnas):
    '''
    Función para calcular los rendimientos (aritméticos) de cada instrumento
    
    ENTRADA:
    datos. Pandas dataframe con los precios de cada instrumento
    
    nombres_columnas. lista con los nombres de las columnas para las que se
    quiere calcular el rendimiento
    
    SALIDA:
    pandas dataframe con los rendimientos de cada instrumento
    '''
    
    #Número de observaciones
    n_obs = datos.shape[0]
    
    #dataframe con los rendimientos
    #la primera fecha no tendrá rendimiento (se omite ya que no hay dato previo)
    rendimientos = pd.DataFrame()
    rendimientos['Date'] = datos['Date'][1:n_obs]
    
    #Reinicia los índices
    rendimientos = rendimientos.reset_index(drop = True)
    
    #Calcula el rendimiento de cada columna
    for columna in nombres_columnas:
        
        #revisa que no haya un precio en 0 o negativo
        if np.all(datos[columna] > 0):
            
            #Ojo con el reset index
            numerador = datos[columna][1:n_obs]
            numerador = numerador.reset_index(drop = True)
            denominador = datos[columna][0:(n_obs - 1)]
            denominador = denominador.reset_index(drop = True)
            
            rendimientos[columna] = numerador / denominador - 1
            
            #normaliza
            media = np.mean(rendimientos[columna])
            desv_est = np.std(rendimientos[columna])
            rendimientos[columna] = (rendimientos[columna] - media) / desv_est
            
        else:
            print('ERROR')
            print('Para la columna', columna,'se tiene un valor de cero o negativo en alguna fecha')
            return 0
    
    return rendimientos

In [3]:
rendimientos = calcula_rendimientos(datos, nombres_columnas)
rendimientos.head(10)

,Date,dow30,naftrac,nasdaq,russell2000,sp500
0,2014-08-08,1.264602,-0.140867,0.749403,0.948610,1.298415
1,2014-08-11,0.062564,1.564473,0.620659,0.853986,0.276291
2,2014-08-12,-0.115822,0.093718,-0.321530,-0.770558,-0.235946
3,2014-08-13,0.588799,-0.009838,0.937384,0.699631,0.736287
4,2014-08-14,0.380130,0.119466,0.357964,0.097360,0.461139
5,2014-08-15,-0.401105,-0.423152,0.204937,-0.172139,-0.052393
6,2014-08-18,1.172998,0.586479,0.887951,1.358057,0.948882
7,2014-08-19,0.506595,0.325453,0.358264,0.300806,0.537442
8,2014-08-20,0.357996,0.633154,-0.077195,-0.436365,0.243477
9,2014-08-21,0.362151,0.424990,0.065402,0.174190,0.298484


In [4]:
#Función para calcular los componentes principales
def pca(datos, nombres_columnas):
    '''
    Función realizar el PCA
    
    ENTRADA:
    datos. Pandas dataframe con los precios de cada instrumento
    
    nombres_columnas. lista con los nombres de las columnas para las que se
    quiere calcular el rendimiento
    
    SALIDA:
    mat_comp_prin. Pandas dataframe que representa la matriz de componentes
    principales
    
    eig_vect. numpy array que representa la matriz cuya i-ésima columna es
    el vector característico correspondiente al i-ésimo valor característico
    
    eig_vals. numpy array que contiene los valores característicos ordenados
    de mayor a menor
    '''
      
    
    #calcula los rendimientos
    rendimientos = calcula_rendimientos(datos, nombres_columnas)
    
    #calcula la matriz de correlaciones
    #Se necesita omitir la columna Date
    #OJO con el parámetro rowvar
    mat_cor = np.corrcoef(rendimientos[nombres_columnas], rowvar = False)
    
    #Calcula valores y vectores característicos de la matriz de correlaciones
    #ver ayuda de np.linalg.eig
    eig_vals, eig_vect = np.linalg.eig(mat_cor)
    
    #Ordena los valores característicos de mayor a menor
    indices_orden = eig_vals.argsort()[::-1]
    eig_vals = eig_vals[indices_orden]
    eig_vect = eig_vect[:, indices_orden]
    
    #calcula la matriz de componentes principales
    #por conveniencia es un pandas dataframe
    mat_comp_prin = np.matmul(datos[nombres_columnas], eig_vect)
    
    #renombra columnas
    nombres_mat_com = ['PC' + str(i + 1) for i in range(0, mat_comp_prin.shape[1])]
    mat_comp_prin.columns = nombres_mat_com
    
    return mat_comp_prin, eig_vect, eig_vals

In [5]:
mat_comp_prin, eig_vect, eig_vals = pca(datos, nombres_columnas)

mat_comp_prin.head(10)

,PC1,PC2,PC3,PC4,PC5
0,11285.156661,3436.168000,-8339.693580,4679.425523,7810.551158
1,11406.356592,3473.078748,-8435.936845,4742.659297,7894.615088
2,11436.095010,3482.813941,-8436.267427,4729.829289,7908.047106
3,11420.249682,3477.355823,-8437.878515,4731.728785,7901.942987
4,11495.253088,3501.272207,-8482.016852,4742.201157,7950.237085
5,11538.401884,3514.544942,-8515.824471,4756.853288,7979.411516
6,11519.191026,3509.701377,-8488.948066,4722.721051,7958.607795
7,11639.110039,3546.469022,-8573.909421,4777.349350,8044.757356
8,11693.312824,3562.935761,-8616.486979,4801.679909,8082.242911
9,11721.253577,3570.493598,-8654.227074,4828.768705,8106.718059


In [6]:
#variación capturada por cada componente principal
variacion_total = eig_vals.sum()
for i in range(0, len(eig_vals)):
    print('El componente', str(i + 1), 'explica', str(eig_vals[i] / variacion_total), "de la variación total")
    
#variación total acumulada
eig_vals.cumsum() / variacion_total

El componente 1 explica 0.8012202601399903 de la variación total
El componente 2 explica 0.13410249499516927 de la variación total
El componente 3 explica 0.0412760831357319 de la variación total
El componente 4 explica 0.020652356112967056 de la variación total
El componente 5 explica 0.002748805616141723 de la variación total


array([0.80122026, 0.93532276, 0.97659884, 0.99725119, 1.        ])